In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
import json
import os
from models.all_model import *
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping
from datetime import datetime

In [ ]:
#config_model 

network="alexnet" #we have 4 options: alexnet,vgg,googlenet,resnet
pretrained=False
dataset="RVL"
n_classes=16
num_EPOCHS = 10
batch_size = 32


In [ ]:
#init model
if network == "vgg":
    IMG_SIZE = 224
    model = vgg(n_classes,IMG_SIZE,dataset,pretrained)
    
elif network == "googlenet" :
    IMG_SIZE = 224
    model = googlenet(n_classes,IMG_SIZE,dataset,pretrained)
elif network == "resnet" :
    IMG_SIZE = 224
    model = resnet(n_classes,IMG_SIZE,dataset,pretrained)
elif network == "alexnet" : 
    IMG_SIZE = 227
    model = Alexnet(n_classes,IMG_SIZE,dataset,pretrained)
model.summary()
# optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])



In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
validation_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)
#Create datagerator for training and testing
train_generator = train_datagen.flow_from_directory(
    './datasets/train',
    target_size=(IMG_SIZE, IMG_SIZE),shuffle=True,
    batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory(
    './datasets/val',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size)

test_generator = test_datagen.flow_from_directory(
    './datasets/test',
       target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size)

label_map = (train_generator.class_indices)


file = open("labelmap.txt", "w")
file.write(json.dumps(label_map))
file.close()

#create checkpoint folder for each model
save_dir = "./checkpoint_"+"RVL_CDIP_"+network+("_pretrained" if pretrained else "_no_pretrained")
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
save_path = save_dir + "/"  + 'weights-{epoch:02d}-{val_acc:.2f}.hdf5'
#model checkpoint, save model with highest validation accuracy
check_pointer = ModelCheckpoint(save_path, monitor='val_acc', mode='max', save_best_only=True)
es = EarlyStopping(monitor='val_acc', mode='max',patience=4)

model.fit_generator(
    train_generator,
    steps_per_epoch=320000// batch_size,
    epochs=num_EPOCHS,
    callbacks=[check_pointer,es],
    validation_data=validation_generator,
    validation_steps=40000 // batch_size)


In [ ]:
#model.load_weights("./googlenet/first_try_cnn_1.hdf5")
model.fit_generator(
    train_generator,
    steps_per_epoch=320000 // batch_size,
    epochs=5,
    callbacks=[check_pointer,es],
    validation_data=validation_generator,
    validation_steps=40000 // batch_size)
